In [1]:
# install openCV package
!pip3 install opencv-python-headless

# To get the average frame count
import glob
import numpy as np
import cv2
import os

# Update the path to your desired directory
video_directory = 'F:/Master/Research Project/Datasets/DFDC/Experiment Dataset/dfdc_train_part_1_exp'

# To get the average frame count 
video_files =  glob.glob(video_directory + '/*.mp4')
video_files = [os.path.normpath(file) for file in video_files]

frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

print("frames", frame_count)
print("Total number of videos:", len(frame_count))
print('Average frame per video:', np.mean(frame_count))

frames [299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 300, 299, 300, 299, 299, 299, 299, 300, 300, 299, 299, 299, 299, 299, 299, 300, 299, 300, 299, 300, 299, 299, 299, 299, 300, 299, 299, 299, 299, 300, 300, 299, 299, 299, 299, 300, 299, 299, 299, 299, 299, 300, 299, 299, 299, 300, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 300, 299, 300, 299, 299, 300, 299, 300, 300, 299, 299, 299, 299, 299, 299, 299, 299, 299, 300, 300, 300, 299, 300, 299, 299, 299, 299, 300, 299, 299, 299, 299, 300, 299, 299, 299, 299, 299, 299, 300, 299, 299, 299, 300, 299, 299, 300, 299, 300, 299, 299, 299, 299, 299, 299, 300, 299, 299, 299, 299, 300, 299, 299, 299, 299, 300, 299, 300, 299, 300, 299, 299, 299, 299, 299, 300, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 300, 299, 299, 299, 299, 299, 300, 300, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 300, 299, 299, 299, 300, 299, 299, 299, 299, 299, 299, 300, 299, 299, 300, 299, 299, 299, 29

In [2]:
!pip3 install facenet-pytorch
!pip3 install tqdm

In [3]:
# to extract frame into images
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = 1
    while success:
        success, image = vidObj.read()
        if success:
            yield image

In [ ]:
!nvidia-smi

In [5]:
from tqdm.autonotebook import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from PIL import Image, ImageDraw
from IPython import display


# Determine if an nvidia GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on device: {}".format(device))

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

# create a face detection pipeline using MTCNN
# mtcnn = MTCNN(keep_all=True, image_size=224, margin=0, device=device)
mtcnn = MTCNN(keep_all=True, image_size=224, margin=0)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained="vggface2").eval()

Running on device: cuda:0
True
1
NVIDIA GeForce RTX 3060


In [6]:
frames_tracked = []
aligned = []



# to create face images
def create_face_images(path_list, out_dir):
    already_present_count = glob.glob(out_dir + "*.jpg")
    print("No of images already present ", len(already_present_count))
    # print(path_list)
    # print(out_dir)

    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.splitext(os.path.basename(path))[0])
        file_exists = glob.glob(out_path + "*.jpg")
        # print(file_exists)
        if len(file_exists) != 0:
            print("Files Already exist: ", out_path)
            continue
        frames = [
            Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            for frame in frame_extract(path)
        ]

        for idx, frame in enumerate(frames):
            print(frame)
            if idx <= 150:
                # Detect faces using MTCNN
                boxes, probs = mtcnn.detect(frame)
                print(boxes)
                print(probs)
                aligned.append(boxes)

                # Draw faces
                frame_draw = frame.copy()
                draw = ImageDraw.Draw(frame_draw)

                # Add to frame list
                frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))

                for i, face in enumerate(boxes):
                    bbox = face
                    # print(bbox)
                    top, right, bottom, left = bbox
                    print(top, right, bottom, left)

                    draw.rectangle(bbox.tolist(), outline=(255, 0, 0), width=6)
                    cropped_image = frame.crop(tuple(bbox))

                    # Extract the face
                    # face_image = frame[top:bottom, left:right, :]
                    # face_image = frame[right : right + left, top : top + bottom]

                    # Save the face as an image
                    face_filename = f"{out_path}_{idx}_face_{i}.jpg"
                    # cv2.imwrite(face_filename, face_image)
                    cropped_image.save(face_filename)

In [7]:
create_face_images(
    video_files, "F:/Master/Research Project/Generated Data/DFDC_Face_only_data"
)

No of images already present  0


  0%|          | 0/1699 [00:00<?, ?it/s]

<PIL.Image.Image image mode=RGB size=1920x1080 at 0x28A138CDA80>
[[518.234619140625 261.7532043457031 790.2434692382812 621.11572265625]
 [832.7639770507812 149.44363403320312 963.9954833984375
  304.53179931640625]]
[0.7906476855278015 0.979488730430603]
518.234619140625 261.7532043457031 790.2434692382812 621.11572265625
832.7639770507812 149.44363403320312 963.9954833984375 304.53179931640625
<PIL.Image.Image image mode=RGB size=1920x1080 at 0x28A138CE530>
[[836.3551635742188 149.27163696289062 966.0438842773438
  300.9661865234375]]
[0.9832161068916321]
836.3551635742188 149.27163696289062 966.0438842773438 300.9661865234375
<PIL.Image.Image image mode=RGB size=1920x1080 at 0x28A138CE410>
[[835.569091796875 150.0800018310547 965.09130859375 303.7608337402344]]
[0.9923816323280334]
835.569091796875 150.0800018310547 965.09130859375 303.7608337402344
<PIL.Image.Image image mode=RGB size=1920x1080 at 0x28A138CDC60>
[[831.8232421875 145.75489807128906 965.2982177734375 309.461181640625

  0%|          | 0/1699 [00:06<?, ?it/s]

[[818.9473876953125 146.16432189941406 958.5640869140625
  324.52606201171875]]
[0.9973716735839844]
818.9473876953125 146.16432189941406 958.5640869140625 324.52606201171875
<PIL.Image.Image image mode=RGB size=1920x1080 at 0x28A138CE7A0>
[[816.6475830078125 146.7167205810547 955.87744140625 323.7806701660156]]
[0.9993502497673035]
816.6475830078125 146.7167205810547 955.87744140625 323.7806701660156
<PIL.Image.Image image mode=RGB size=1920x1080 at 0x28A138CE650>


KeyboardInterrupt: 

In [28]:
def calculate_embedding(path):
    img = Image.open(path)
    # Get cropped and prewhitened image tensor
    img_cropped = mtcnn(img)
    # Calculate embedding (unsqueeze to add batch dimension)
    img_embedding = resnet(img_cropped)
    print(img_embedding)

In [29]:
calculate_embedding("F:\Master\Research Project\Generated Data\DFDC_Face_only_data\\aassnaulhq_0_face_1.jpg")

tensor([[ 0.0185,  0.0610, -0.0175,  0.0180, -0.0069, -0.0006, -0.0789,  0.0291,
          0.0499,  0.0151, -0.0404,  0.0242, -0.0472, -0.0576,  0.0560,  0.0413,
          0.0925,  0.0263,  0.0467, -0.0192,  0.0329,  0.0220,  0.0178,  0.0354,
         -0.0150,  0.0540,  0.0393,  0.0331,  0.0079,  0.0522, -0.0300,  0.0391,
          0.0026,  0.0156, -0.0275,  0.0024,  0.0091,  0.0028, -0.0062,  0.0398,
          0.0659, -0.0257, -0.0253,  0.0576, -0.0578, -0.0390, -0.0011,  0.0350,
         -0.1192, -0.0072, -0.0280,  0.0353,  0.0183, -0.0460, -0.0412,  0.0268,
          0.0168,  0.0198,  0.0812,  0.0049, -0.0295,  0.0719, -0.0520, -0.0298,
         -0.0192,  0.0465, -0.0334,  0.0291, -0.0380, -0.0025,  0.0680, -0.0478,
          0.0102,  0.0113,  0.0242,  0.0213, -0.1114,  0.0629, -0.0384, -0.0510,
          0.0802, -0.0413, -0.0689,  0.0344, -0.0203, -0.0769, -0.0447,  0.1119,
         -0.0433, -0.0188, -0.0376,  0.0331, -0.0069,  0.0328,  0.0414, -0.0675,
         -0.0619, -0.0006, -

In [ ]:
d = display.display(frames_tracked[0], display_id=True)
i = 1
try:
    while True:
        d.update(frames_tracked[i % len(frames_tracked)])
        i += 1
except KeyboardInterrupt:
    pass